## Import

In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [ ]:
%%capture
!pip install rouge_score
!pip install git+https://github.com/k4black/codebleu.git
!pip install tree-sitter-python

## Base Model

In [ ]:
from unsloth import FastLanguageModel
import torch
from google.colab import userdata
HF_TOKEN = userdata.get('HF_TOKEN')
# Memuat model dasar (base model)
model_base, tokenizer_base = FastLanguageModel.from_pretrained(
    model_name="Qwen/Qwen2.5-7B",  # Model dasar (base model) yang belum di-fine-tune
    max_seq_length=2048,
    dtype=None,
    load_in_4bit=True,
    token=userdata.get('HF_TOKEN')
)
FastLanguageModel.for_inference(model_base)  # Mengaktifkan inferensi yang lebih cepat

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch SmolVLMForConditionalGeneration forward function.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.4.1: Fast Qwen2 patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 3584, padding_idx=151654)
    (layers): ModuleList(
      (0-1): 2 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=3584, out_features=3584, bias=True)
          (k_proj): Linear(in_features=3584, out_features=512, bias=True)
          (v_proj): Linear(in_features=3584, out_features=512, bias=True)
          (o_proj): Linear(in_features=3584, out_features=3584, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear4bit(in_features=3584, out_features=18944, bias=False)
          (up_proj): Linear4bit(in_features=3584, out_features=18944, bias=False)
          (down_proj): Linear4bit(in_features=18944, out_features=3584, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)

## Lora Model

In [ ]:
from unsloth import FastLanguageModel
import torch
from google.colab import userdata
HF_TOKEN = userdata.get('HF_TOKEN')

# Memuat model LoRA yang sudah di-fine-tune
model_lora, tokenizer_lora = FastLanguageModel.from_pretrained(
    model_name="ncardian/Qwen2.5-7B-data-coder-LoRA",  # Model LoRA yang sudah di-fine-tune
    max_seq_length=2048,
    dtype=None,
    load_in_4bit=True,
    token=userdata.get('HF_TOKEN')
)
FastLanguageModel.for_inference(model_lora)  # Mengaktifkan inferensi yang lebih cepat

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch SmolVLMForConditionalGeneration forward function.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.4.1: Fast Qwen2 patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Unsloth 2025.4.1 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(152064, 3584, padding_idx=151654)
        (layers): ModuleList(
          (0-1): 2 x Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=3584, out_features=3584, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3584, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=3584, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear(
 

## Dataset

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer_lora.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset_path = "/content/drive/MyDrive/Lora/dataset-pd-coder.json"

dataset = load_dataset("json", data_files=dataset_path)
dataset = dataset["train"].train_test_split(test_size=0.06)
dataset = dataset.map(formatting_prompts_func, batched = True,)
dataset

Map:   0%|          | 0/4700 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'text'],
        num_rows: 4700
    })
    test: Dataset({
        features: ['instruction', 'input', 'output', 'text'],
        num_rows: 300
    })
})

## Quality Base Model Test

In [ ]:
from transformers import TextStreamer
from unsloth import FastLanguageModel

# Aktifkan mode inference 2x lebih cepat
FastLanguageModel.for_inference(model_base)

# Template prompt ala Alpaca
alpaca_prompt = (
    "### Instruction:\n{instruction}\n\n"
    "### Input:\n{input}\n\n"
    "### Response:\n{output}"
)

# Prompt generation
prompt = alpaca_prompt.format(
    instruction="Show plot No2 vs So2?",
    input='{ "Date": "str", "Wspeed": "int", "NO2: "float", "wd": "str", "SO2": "float" }',
    output=""  # kosongkan untuk prompting
)

# Tokenisasi
inputs = tokenizer_base([prompt], return_tensors="pt").to("cuda")

# Stream output
text_streamer = TextStreamer(tokenizer_base)
_ = model_base.generate(
    **inputs,
    streamer=text_streamer,
    max_new_tokens=1024,
    do_sample=True,
    temperature=0.2
)


### Instruction:
Show plot No2 vs So2?

### Input:
{ "Date": "str", "Wspeed": "int", "NO2: "float", "wd": "str", "SO2": "float" }

### Response:
To show the plot of SO2 vs NO2, you can use the matplotlib library in Python. The code would look something like this:

```python
import pandas as pd
import matplotlib.pyplot as plt

# Load the data into a dataframe
data = pd.DataFrame({ "Date": "str", "Wspeed": "int", "NO2": "float", "wd": "str", "SO2": "float" })

# Plot the data
plt.scatter(data['NO2'], data['SO2'])
plt.xlabel('NO2')
plt.ylabel('SO2')
plt.title('Plot of SO2 vs NO2')
plt.show()
```<|endoftext|>


## Quality Lora Model Test

In [ ]:
from transformers import TextStreamer
from unsloth import FastLanguageModel

# Aktifkan mode inference 2x lebih cepat
FastLanguageModel.for_inference(model_lora)

# Template prompt ala Alpaca
alpaca_prompt = (
    "### Instruction:\n{instruction}\n\n"
    "### Input:\n{input}\n\n"
    "### Response:\n{output}"
)

# Prompt generation
prompt = alpaca_prompt.format(
    instruction="Show plot No2 vs So2",
    input='{ "Date": "str", "Wspeed": "int", "NO2: "float", "wd": "str", "SO2": "float" }',
    output=""  # kosongkan untuk prompting
)

# Tokenisasi
inputs = tokenizer_lora([prompt], return_tensors="pt").to("cuda")

# Stream output
text_streamer = TextStreamer(tokenizer_lora)
_ = model_lora.generate(
    **inputs,
    streamer=text_streamer,
    max_new_tokens=1024,
    do_sample=True,
    temperature=0.2
)


### Instruction:
Show plot No2 vs So2?

### Input:
{ "Date": "str", "Wspeed": "int", "NO2: "float", "wd": "str", "SO2": "float" }

### Response:
# Import necessary libraries
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

# Assume df is already loaded in memory

# Data cleaning
df = df.dropna()  # Remove missing values
df = df.drop_duplicates()  # Remove duplicates

# Scatter chart visualization
fig = px.scatter(df, x='NO2', y='SO2', 
                 color='Date',
                 title='Show plot No2 vs So2?')
fig.show()
<|endoftext|>


## Benchmark

In [ ]:
from datasets import load_dataset
from tqdm import tqdm
import nltk

# Download the necessary data files for Punkt Tokenizer Model
nltk.download('punkt_tab')
from nltk import word_tokenize
from rouge_score import rouge_scorer

test_data = dataset["test"] #.shuffle(seed=42).select(range(100))

from codebleu import calc_codebleu

alpaca_prompt_template = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Answer the following question truthfully.

### Input:
{}

### Response:
"""

def codebleu_score(pred, target, lang='python'):
    result = calc_codebleu([target], [pred], lang)
    return result['codebleu']


def generate_answer_alpaca_style(question):
    prompt = alpaca_prompt_template.format(question)
    # Tokenize the prompt and convert to tensor
    inputs = tokenizer_lora(prompt, return_tensors="pt").to("cuda")
    # Generate response using the model
    outputs = model_lora.generate(**inputs, max_new_tokens=512, use_cache=True)
    # Decode the response to text
    response = tokenizer_lora.decode(outputs[0], ignore_special_tokens=True)
    return response.strip()

def generate_answer_alpaca_style1(question):
    prompt = alpaca_prompt_template.format(question)
    # Tokenize the prompt and convert to tensor
    inputs = tokenizer_base(prompt, return_tensors="pt").to("cuda")
    # Generate response using the model
    outputs = model_base.generate(**inputs, max_new_tokens=512, use_cache=True)
    # Decode the response to text
    response = tokenizer_base.decode(outputs[0], ignore_special_tokens=True)
    return response.strip()

# Evaluator F1
def f1_word_overlap(pred, target):
    pred_tokens = set(word_tokenize(pred.lower()))
    target_tokens = set(word_tokenize(target.lower()))
    overlap = pred_tokens & target_tokens
    if not target_tokens:
        return 0.0
    precision = len(overlap) / len(pred_tokens) if pred_tokens else 0
    recall = len(overlap) / len(target_tokens)
    if precision + recall == 0:
        return 0.0
    return 2 * precision * recall / (precision + recall)

# Evaluator ROUGE-L
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
def rouge_l_score(pred, target):
    scores = scorer.score(target, pred)
    return scores["rougeL"].fmeasure

# Fungsi utama benchmarking
def benchmark_model(test_data, generate_func):
    results = []
    for item in tqdm(test_data):
        if item['input']:
            question = f"{item['instruction']}\n{item['input']}"
        else:
            question = item['instruction']
        truth = item['output']

        model_answer = generate_func(question)
        f1 = f1_word_overlap(model_answer, truth)
        rouge_l = rouge_l_score(model_answer, truth)
        codebleu = codebleu_score(model_answer, truth)

        results.append({
            "question": question,
            "truth": truth,
            "model_answer": model_answer,
            "f1_score": f1,
            "rouge_l_score": rouge_l,
            "codebleu_score": codebleu
        })

    avg_f1 = sum(r["f1_score"] for r in results) / len(results)
    avg_rouge_l = sum(r["rouge_l_score"] for r in results) / len(results)
    avg_codebleu = sum(r["codebleu_score"] for r in results) / len(results)
    print('\n')
    print(f"\nAverage CodeBLEU Score   : {avg_codebleu:.4f}")
    print(f"Average F1 Overlap Score: {avg_f1:.4f}")
    print(f"Average ROUGE-L Score   : {avg_rouge_l:.4f}")
    return results


## Base

In [ ]:
base_model = benchmark_model(test_data, generate_answer_alpaca_style1)

100%|██████████| 300/300 [48:50<00:00,  9.77s/it]




Average CodeBLEU Score   : 0.1922
Average F1 Overlap Score: 0.1203
Average ROUGE-L Score   : 0.0498


## Lora

In [ ]:
lora_model = benchmark_model(test_data, generate_answer_alpaca_style)

100%|██████████| 300/300 [34:37<00:00,  6.93s/it]




Average CodeBLEU Score   : 0.6522
Average F1 Overlap Score: 0.5522
Average ROUGE-L Score   : 0.4842


In [14]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

# Data
metrics = ['Avg CodeBLEU', 'Avg F1-Overlap', 'Avg ROUGE-L']
metric_descriptions = [
    "Measures similarity between generated and reference code",
    "Evaluates token-level overlap between predictions and references",
    "Measures longest common subsequence between texts"
]
base_scores = [0.1922, 0.1203, 0.0498]
lora_scores = [0.6522, 0.5522, 0.4842]
improvement = [(l/b) for b,l in zip(base_scores,lora_scores)]

# Training metrics

trainvval_desc = "Train loss measures errors on training data, val loss on <br> validation data; a small gap indicates good model learning."
perplexity_desc = "The closer to 1, the more accurate the LLM's predictions."

train_loss = 0.0314
eval_loss = 0.0312
perplexity = 1.03

# Color Palette
COLORS = {
    'dark_bg': '#0f0f13',
    'darker_bg': '#08080b',
    'accent_blue': '#00f5ff',
    'accent_pink': '#ff00f5',
    'accent_purple': '#610C9F',
    'gradient_fire': '#F05941',
    'gradient_ocean': '#1D267D',
    'text': '#ffffff',
    'grid': 'rgba(255, 255, 255, 0.05)',
    'highlight': '#ffd700'
}

# Create figure with custom layout
fig = make_subplots(
    rows=2, cols=2,
    row_heights=[0.7, 0.3],
    column_widths=[0.6, 0.4],
    vertical_spacing=0.15,
    horizontal_spacing=0.15,
    specs=[
        [{"type": "bar", "colspan": 2}, None],
        [{"type": "bar"}, {"type": "bar"}]
    ]
)

# Main Bars (top row)
fig.add_trace(go.Bar(
    x=metrics,
    y=base_scores,
    name='Base Model',
    marker_color=COLORS['accent_blue'],
    marker_line_color='rgba(255,255,255,0.8)',
    marker_line_width=1,
    opacity=0.9,
    text=[f'<b>{score:.3f}</b>' for score in base_scores],
    textposition='outside',
    textfont=dict(color=COLORS['text'], size=14),
    hoverinfo='y+name'
), row=1, col=1)

fig.add_trace(go.Bar(
    x=metrics,
    y=lora_scores,
    name='LoRA Model',
    marker_color=COLORS['accent_pink'],
    marker_line_color='rgba(255,255,255,0.8)',
    marker_line_width=1,
    opacity=0.9,
    text=[f'<b>{score:.3f}</b>' for score in lora_scores],
    textposition='outside',
    textfont=dict(color=COLORS['text'], size=14),
    hoverinfo='y+name'
), row=1, col=1)

# Improvement bars (bottom left)
fig.add_trace(go.Bar(
    x=metrics,
    y=improvement,
    name='Improvement',
    marker_line_width=1,
    marker_color=COLORS['accent_purple'],
    marker_line_color='rgba(255,255,255,0.5)',
    text=[f'<b>{imp:.2f}x</b>' for imp in improvement],
    textposition='inside',
    textfont=dict(color=COLORS['highlight'], size=14, family="Arial Black"),
    hoverinfo='y+name'
), row=2, col=1)

# Training metrics (bottom right)
fig.add_trace(go.Bar(
    x=['Train Loss', 'Eval Loss'],
    y=[train_loss, eval_loss],
    name='Training Metrics',
    marker_color=[COLORS['gradient_fire'], COLORS['gradient_ocean']],
    marker_line_color='rgba(255,255,255,0.8)',
    marker_line_width=1,
    text=[f'<b>{train_loss:.4f}</b>', f'<b>{eval_loss:.4f}</b>'],
    textposition='auto',
    textfont=dict(color=COLORS['text'], size=14),
), row=2, col=2)


# Custom Layout
fig.update_layout(
    height=900,
    template='plotly_dark',
    plot_bgcolor=COLORS['dark_bg'],
    paper_bgcolor=COLORS['darker_bg'],
    title={
        'text': "<b>LoRA Fine Tuning Qwen2.5-7b Evaluation</b>",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': {'size': 28, 'color': COLORS['text'], 'family': "Arial Black"},
    },
    xaxis=dict(
        title='<b>EVALUATION METRICS</b>',
        titlefont=dict(size=18, color=COLORS['text'], family="Arial Black"),
        tickfont=dict(size=14, color=COLORS['text']),
        gridcolor=COLORS['grid'],
        linecolor=COLORS['grid'],
        showgrid=False
    ),
    xaxis2=dict(
        showgrid=False,
        tickfont=dict(size=14, color=COLORS['text'])
    ),
    xaxis3=dict(
        title='<b>TRAINING METRICS</b>',
        titlefont=dict(size=14, color=COLORS['text'], family="Arial Black"),
        tickfont=dict(size=12, color=COLORS['text']),
        gridcolor=COLORS['grid'],
        showgrid=False
    ),
    yaxis=dict(
        title='<b>SCORE</b>',
        titlefont=dict(size=16, color=COLORS['text'], family="Arial Black"),
        tickfont=dict(size=12, color=COLORS['text']),
        gridcolor=COLORS['grid'],
        range=[0, 0.8],
        dtick=0.2
    ),
    yaxis2=dict(
        title='<b>IMPROVEMENT </b>',
        titlefont=dict(size=16, color=COLORS['text'], family="Arial Black"),
        tickfont=dict(size=12, color=COLORS['text']),
        gridcolor=COLORS['grid'],
        range=[0, max(improvement)*1.2]
    ),
    yaxis3=dict(
        title='<b>LOSS</b>',
        titlefont=dict(size=14, color=COLORS['text'], family="Arial Black"),
        tickfont=dict(size=12, color=COLORS['text']),
        gridcolor=COLORS['grid'],
        range=[0, max(train_loss, eval_loss)*1.5]
    ),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="center",
        x=0.5,
        font=dict(size=14, color=COLORS['text'], family="Arial Black"),
        bgcolor='rgba(0,0,0,0)'
    ),
    barmode='group',
    bargap=0.4,
    bargroupgap=0.1,
    margin=dict(l=80, r=80, t=150, b=200),
    hoverlabel=dict(
        bgcolor='rgba(15,15,25,0.9)',
        font_size=14,
        font_family="Arial"
    )
)

# Metric descriptions
for i, (metric, desc) in enumerate(zip(metrics, metric_descriptions)):
    fig.add_annotation(
        x=-1,
        y = {0: -0.07, 1: -0.10, 2: -0.13}.get(i, None),
        xref="x",
        yref="paper",
        text=f"<b>{metric} : </b>{desc}",
        showarrow=False,
        font=dict(size=12, color='rgba(255,255,255,0.8)'),
        xanchor="left",
        yanchor="top",
        align="left"
    )

# Performance highlights
fig.add_annotation(
    x=0.5, y=1.125,
    xref="paper", yref="paper",
    text=f"🔥 LoRA achieves : <span style='color:#ffd700'>{np.round(lora_scores[0]/base_scores[0],1)}x</span> better CodeBLEU, <span style='color:#ffd700'>{np.round(lora_scores[1]/base_scores[1],1)}x</span> better F1, and <span style='color:#ffd700'>{np.round(lora_scores[2]/base_scores[2],1)}x</span> better ROUGE-L, Perplexity: <span style='color:{COLORS['highlight']}'>{perplexity:.2f}</span>",
    showarrow=False,
    font=dict(size=16, color=COLORS['text'], family="Arial"),
    align="center",
    bgcolor="rgba(138, 43, 226, 0.2)",
    bordercolor=COLORS['accent_purple']
)

# Add perplexity as text annotation in the bottom right
fig.add_annotation(
    x=0.6, y=-0.25,
    xref="paper", yref="paper",
    text=f"<b>Perplexity:</b> {perplexity_desc}",
    showarrow=False,
    font=dict(size=12, color='rgba(255,255,255,0.8)'),
    xanchor="left",
    yanchor="top",
    align="left"

)

fig.add_annotation(
    x=0.8, y=-0.11,
    xref="paper", yref="paper",
    text=f"<b>Train Loss vs Val loss:</b><br> {trainvval_desc}",
    showarrow=False,
    font=dict(size=12, color='rgba(255,255,255,0.8)'),
    xanchor="center",
    yanchor="top",
    align="center"

)

# Tech stack and signature
fig.add_annotation(
    x=0.2, y=-0.35,
    xref="paper", yref="paper",
    text="<b>Tech Stack:</b> PyTorch | Unsloth | Transformer | LoRA (Low-Rank Adaptation) | Plotly",
    showarrow=False,
    font=dict(size=12, color='rgba(255,255,255,0.7)'),
    xanchor="center"
)

# Tech stack and signature
fig.add_annotation(
    x=0.22, y=-0.27,
    xref="paper", yref="paper",
    text="<b>40 million</b> Trainable parameters (0.58% from 7 billion parameters)",
    showarrow=False,
    font=dict(size=12, color='rgba(255,255,255,0.7)'),
    xanchor="center"
)
fig.add_annotation(
    x=0.215, y=-0.24,
    xref="paper", yref="paper",
    text="<b>Datasets</b> 4700 rows training, 300 rows validation, alpaca format",
    showarrow=False,
    font=dict(size=12, color='rgba(255,255,255,0.7)'),
    xanchor="center"
)

fig.add_annotation(
    x=0, y=1,
    xref="paper", yref="paper",
    text="higher score = better performance.",
    showarrow=False,
    font=dict(size=12, color='rgba(255,255,255,0.8)'),
    xanchor="left",
    yanchor="top",
    align="left"
)

fig.add_annotation(
    x=1.05, y=-0.35,
    xref="paper", yref="paper",
    text="🔗 @ardiannurcahya | #AIModels #ParameterEfficientFineTuning",
    showarrow=False,
    font=dict(size=12, color='rgba(255,255,255,0.7)'),
    xanchor="right"
)

# Add glow effects
for i in range(len(metrics)):
    fig.add_shape(type="rect",
        xref="x", yref="y",
        x0=i-0.4, y0=0,
        x1=i+0.4, y1=max(lora_scores),
        line=dict(width=0),
        fillcolor=COLORS['accent_pink'],
        opacity=0.03,
        layer="below"
    )

# Add glow effects
for i in range(len(metrics)):
    fig.add_shape(type="rect",
        xref="x", yref="y",
        x0=i-0.4, y0=0,
        x1=i+0.4, y1=max(lora_scores),
        line=dict(width=0),
        fillcolor=COLORS['accent_pink'],
        opacity=0.03,
        layer="below"
    )

# Apply rounded corners to all bars
for trace in fig.data:
    if trace.type == 'bar':
        trace.marker.line.width = 1.5
        trace.marker.cornerradius = 5

fig.show()